In [29]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn import preprocessing, pipeline, ensemble, compose
import datasets
import os

# cols = ['age', 'workclass', 'fnlwgt', 'income']

real_path = '/hdd3/sonia/data/adult.csv'
dgpt2_path = '/hdd3/sonia/be_great/ckpts/dgpt2/adult-allcol/samples.csv'
moe_path = '/hdd3/sonia/be_great/ckpts/moe/dgpt2/adult-allcol/jul21/samplesclean.csv'
greatdpt2_path = '/hdd3/sonia/be_great/ckpts/dgpt2-greatclean.csv'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
real = pd.read_csv(real_path)#[cols]
dgpt2= pd.read_csv(dgpt2_path)
moe  = pd.read_csv(moe_path)
greatdgpt2=pd.read_csv(greatdpt2_path)
print(real.shape, dgpt2.shape, moe.shape, greatdgpt2.shape)

rs = 2
min_dataset_size = min(len(real), len(dgpt2), len(moe), len(greatdgpt2))
real = real.sample(min_dataset_size, random_state=rs)
dgpt2=dgpt2.sample(min_dataset_size, random_state=rs)
moe  =  moe.sample(min_dataset_size, random_state=rs)
greatdgpt2 = greatdgpt2.sample(min_dataset_size, random_state=rs)
real.shape, dgpt2.shape, moe.shape, greatdgpt2.shape

(48842, 15) (5935, 15) (9106, 15) (9815, 15)


((5935, 15), (5935, 15), (5935, 15), (5935, 15))

In [31]:
datadict = {'real':real, 'dgpt2':dgpt2, 'moe':moe, 'greatdgpt2':greatdgpt2}
datadict.keys()

dict_keys(['real', 'dgpt2', 'moe', 'greatdgpt2'])

In [32]:
ords = ['workclass', 'education', 'marital-status', 'occupation', 
        'relationship', 'race', 'sex', 'native-country', 'income'] # MUST BE IN ORDER
nums = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', ]
labs = ['income']

# for each ord column, get all unique values occurign in real/synth, train/test
categories = []
for name in ords:
    s = set(real[name].unique().tolist())
    s.update(dgpt2[name].unique().tolist())
    s.update(moe[name].unique().tolist())
    s.update(greatdgpt2[name].unique().tolist())
    
    categories.append( list(s) )
categories

ordenc = preprocessing.OrdinalEncoder(categories=categories)
numenc = preprocessing.StandardScaler()
lb = preprocessing.LabelBinarizer()

In [75]:
def create_pipeline(trainset):
    rfc = ensemble.RandomForestClassifier(n_estimators=10, max_depth=4)
    preprocessing_pipeline = compose.ColumnTransformer([
        ("ordinal_preprocessor", ordenc, ords),
        ("numerical_preprocessor", numenc, nums),
    ])
    complete_pipeline = pipeline.Pipeline([
        ("preprocessor", preprocessing_pipeline),
        ("estimator", rfc)
    ])
    
    preprocessed_labels = lb.fit_transform(trainset[labs].values.ravel()).ravel()
    complete_pipeline.fit(trainset[ords+nums], preprocessed_labels)
    return complete_pipeline

In [76]:
# %%capture
# create random forest pipelines
rfdict = {}
for src in datadict.keys():
    print(src)
    rfdict[src] = create_pipeline(datadict[src])

real
dgpt2
moe
greatdgpt2


In [77]:
for data in datadict.keys():
    labels = lb.fit_transform(datadict[data][labs])
    for model in rfdict.keys():
        score = rfdict[model].score(datadict[data][ords+nums], labels)
        print(f'{model} on {data}: \t\t\t{score}')

real on real: 			0.9826453243470935
dgpt2 on real: 			0.010278011794439764
moe on real: 			0.27363100252737993
greatdgpt2 on real: 			0.32299915754001685
real on dgpt2: 			0.3467565290648694
dgpt2 on dgpt2: 			0.9976411120471778
moe on dgpt2: 			0.9816343723673125
greatdgpt2 on dgpt2: 			0.9511373209772536
real on moe: 			0.3363100252737995
dgpt2 on moe: 			0.998989048020219
moe on moe: 			0.9939342881213142
greatdgpt2 on moe: 			0.9755686604886268
real on greatdgpt2: 			0.4306655433866891
dgpt2 on greatdgpt2: 			1.0
moe on greatdgpt2: 			0.9962931760741365
greatdgpt2 on greatdgpt2: 			0.9885425442291491
